In [1]:
#regualr spacy instal lwould not work, had to get from github
#conda install -c conda-forge spacy
#python -m spacy download en
#check with: python -m spacy validate
#restart jupyter notebook

import codecs

import spacy
import pandas as pd
import itertools as it
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

import en_core_web_sm
#nlp = en_core_web_sm.load()
#import en_core_web_md
#nlp = en_core_web_md.load()
nlp = spacy.load('en')

In [2]:
#test
with codecs.open('review_text_lv_rest_subset_30.txt', encoding='utf_8') as f:
    sample_review = list(it.islice(f, 8, 9))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print(sample_review)

I live very near this place and have been curious to try it for sometime now as it always seems busy.

Well my curiosity has been satisfied and I'm sure I'll never feel the need to visit this place again.

It struck me as a typical trendy place that yuppies feel they need to visit as all the other yuppies do.  
"Oh the food is sooo good!!"
"Really? You've eaten there?"
"No. But all my yuppy friends have and they say it's good so it must be."

I only give it a two because the service and presentation of the food was decent.

We ordered something called oxtail fries which were horrible. I'm sure an actual oxtail would probably have been tastier.

My girlfriend and I each ordered a different burger so we could sample each others. Sorry I can't remember what they were called but we didn't care for either one. The best compliment I could pay them is to say they were at least edible.

So as mentioned the service was decent and the food looked presentable but as far as taste I can't recommend

In [3]:
#helper functions from modern nlp in python
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [4]:
unigram_sentences_filepath = 'unigram_sentences_all_eat_30.txt'

In [6]:
%%time
if 1 == 0:
    with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus('review_text_eats_subset_30.txt'):
            f.write(sentence + '\n')

CPU times: user 2d 14h 54min 22s, sys: 1d 16h 18min 7s, total: 4d 7h 12min 29s
Wall time: 15h 9min 21s


In [7]:
%%time
if 1 == 0:
    with codecs.open('unigram_sentences_all_lv_rest_30.txt', 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus('review_text_lv_rest_subset_30.txt'):
            f.write(sentence + '\n')

CPU times: user 17h 53min 55s, sys: 12h 6min 47s, total: 1d 6h 43s
Wall time: 4h 29min 35s


In [6]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

#build models with larger dataset

In [7]:
for unigram_sentence in it.islice(unigram_sentences, 230, 240):
    print( u' '.join(unigram_sentence))
    print(u'')

at first -PRON- be not impress that -PRON- be

a 45 min wait however -PRON- bf really want to try this piece so -PRON- decide to wait

-PRON- be definitely different and worth the wait

-PRON- start off with the house cured meat -PRON- be good but definitely too much for just 2 people to share and no need to order the extra bread if -PRON- be only two people

next come the roasted bone marrow which be amazing and definitely worth ordering

have -PRON- not order so many other dish -PRON- would have order a second round of this

who know bone marrow could be this tasty

then come the spicy horse tartare which -PRON- think be intriguing as -PRON- have never hear so -PRON- have order -PRON-

when -PRON- show up -PRON- be like a salad

be not bad worth try but probably only a one time thing for -PRON-



In [8]:
bigram_model_filepath = 'bigram_model_all_eat_30'


In [11]:
%%time

#time consuming
if 0 == 1:

    bigram_model = Phrases(unigram_sentences)

    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

CPU times: user 8min 49s, sys: 5.43 s, total: 8min 54s
Wall time: 8min 54s


In [9]:
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

In [10]:
bigram_sentences_filepath = 'bigram_sentences_all_eat_30.txt'

In [14]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 0:

    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 19min 41s, sys: 5.7 s, total: 19min 47s
Wall time: 19min 50s


In [11]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [12]:
for bigram_sentence in it.islice(bigram_sentences, 130, 170):
    print(u' '.join(bigram_sentence))
    print(u'')    

but the roti -PRON- be really good

most of the dish be under $ 10 definitely a good deal

will be back to try the noodle dish

if -PRON- have not yet try wasabi do not bother

-PRON- be expensive and the food be disgusting

-PRON- have to order the sushi at the table but everything else i.e. salad soup cook food -PRON- can get up and go get -PRON- -PRON-

the food be very bland the sushi do not taste like anything not even sure if the fish be fresh

on the weekend everyone seem to be there for the snow_crab leg if that be what -PRON- be come for forget -PRON-

-PRON- have to fight wait in line to get -PRON-

-PRON- be probably easy to eat -PRON- at mandarin

if anyone suggest to go there -PRON- highly_recommend -PRON- go somewhere_else

decide to try this place after read the review

be not disappoint

huge portion and a bit of a long wait

-PRON- will take phone order over the walk in

so call_ahead to order

try the burrito with rice but will be back to try the noodle version

hide 

In [13]:
trigram_model_filepath = 'trigram_model_all_eat_30'

In [14]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 1 == 0:

    trigram_model = Phrases(bigram_sentences)

    trigram_model.save(trigram_model_filepath)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 11 µs


In [15]:
trigram_model = Phrases.load(trigram_model_filepath)

In [16]:
trigram_sentences_filepath = 'trigram_sentences_all_eat_30.txt'

In [17]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 0:

    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            
            trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            
            f.write(trigram_sentence + '\n')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


In [18]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [19]:
for trigram_sentence in it.islice(trigram_sentences, 320, 520):
    print(u' '.join(trigram_sentence))
    print(u'')

stumble_upon this one day thought -PRON- would give -PRON- a try

typical sushi place sushi be pretty good

price be decent

party tray be decent

=)

fish be good really good

love how -PRON- could have so many different option for fish choice

chip be only ok

onion_ring be not the same as the regular onion a little too crispy

-PRON- have order the house organic batter mahi_mahi very light not really heavy

creamy coleslaw be flavourless

share the o.t.h poutine which be ok but not for -PRON-

will go back to try the pull_pork or duck_confit one

good for gluten_free

=)

after read the review on yelp -PRON- boyfriend and i decide to give this place a try especially since -PRON- be on a mission to try raman in every city -PRON- visit

-PRON- be really excited as when -PRON- be in nyc -PRON- have really good raman

listen to the review -PRON- start off with the gyoza appetizer which be not as good as everyone say -PRON- be

-PRON- be pan_fry with the thin_layer of skin but there be a

In [20]:
#Write the transformed text out to a new file, with one review per line.

trigram_reviews_filepath = 'trigram_transformed_reviews_eats.txt'



In [21]:
review_txt_filepath = 'review_text_eats_subset_30.txt'

In [40]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 0:

    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                      batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in spacy.lang.en.stop_words.STOP_WORDS]
            
            # write the transformed review as a line in the new file
            trigram_review = u' '.join(trigram_review)
            f.write(trigram_review + '\n')

/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 2d 15h 54min 4s, sys: 1d 17h 14min 56s, total: 4d 9h 9min 1s
Wall time: 15h 58min 46s


In [41]:
#spacy.lang.en.stop_words.STOP_WORDS

In [22]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import _pickle as pickle

In [24]:
trigram_dictionary_filepath = 'trigram_dict_eat_30.dict'

In [44]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if 1 == 0:

    trigram_reviews = LineSentence(trigram_reviews_filepath)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

CPU times: user 2min 24s, sys: 506 ms, total: 2min 24s
Wall time: 2min 24s


In [25]:
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

In [29]:
trigram_reviews = LineSentence(trigram_reviews_filepath)

In [31]:
trigram_dictionary = Dictionary(trigram_reviews)

In [32]:
len(trigram_dictionary)

584041

In [25]:
trigram_bow_filepath = 'trigram_bow_corpus_eat_30.mm'

In [26]:

def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [47]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if 1 == 1:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_reviews_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

CPU times: user 3min 38s, sys: 6.2 s, total: 3min 44s
Wall time: 3min 45s


In [27]:
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

In [28]:
lda_model_filepath = 'lda_model_eat_30'

In [49]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if 1 == 1:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=50,
                           id2word=trigram_dictionary,
                           workers=3)
    
    lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

CPU times: user 34min 53s, sys: 2min 36s, total: 37min 30s
Wall time: 35min 16s


In [29]:
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

In [30]:
def explore_topic(topic_number, topn=50):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print(u'{:25} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda.show_topic(topic_number, topn=50):
        print(u'{:25} {:.3f}'.format(term, round(frequency, 3)))

In [31]:
explore_topic(topic_number = 4)

term                      frequency

sushi                     0.092
roll                      0.074
fish                      0.022
fresh                     0.019
salmon                    0.018
order                     0.018
japanese                  0.016
rice                      0.013
sashimi                   0.013
come                      0.012
eat                       0.011
tuna                      0.011
try                       0.011
like                      0.010
ayce                      0.009
chef                      0.008
quality                   0.008
piece                     0.007
tempura                   0.007
menu                      0.007
great                     0.007
time                      0.007
restaurant                0.006
sake                      0.006
spicy                     0.006
service                   0.005
definitely                0.005
taste                     0.005
miso_soup                 0.005
pretty                    0.005
spe

In [32]:
topic_names = {0: u'chinese',
               1: u'thai',
               2: u'healthy',
               3: u'smell',
               4: u'japanese',
               5: u'toronto',
               6: u'service',
               7: u'experience',
               8: u'asian soup',
               9: u'grocery',
               10: u'parking',
               11: u'bar ambiance',
               12: u'uk',
               13: u'good service',
               14: u'fun ambiance',
               15: u'young',
               16: u'comfort food',
               17: u'greek',
               18: u'high end',
               19: u'hotwing',
               20: u'breakfast',
               21: u'sweet',
               22: u'wine & dine',
               23: u'pubs',
               24: u'good taste',
               25: u'na drinks',
               26: u'desserts',
               27: u'coffee shop',
               28: u'mexican',
               29: u'reviews',
               30: u'new york',
               31: u'general restaurant',
               32: u'beach',
               33: u'location',
               34: u'happy hour',
               35: u'amazing',
               36: u'vietnamese',
               37: u'time',
               38: u'vas legas',
               39: u'montreal',
               40: u'deli',
               41: u'buffet',
               42: u'bbq',
               43: u'french',
               44: u'money',
               45: u'street taco',
               46: u'pizza',
               47: u'airport & delivery',
               48: u'burger & fries',
               49: u'italian'}

In [33]:
topic_names_filepath = 'topic_names_eats_30.pkl'

with open(topic_names_filepath, 'wb') as f:
    pickle.dump(topic_names, f)

In [34]:
LDAvis_data_filepath ='ldavis_prepared_eats_30'


In [108]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                              trigram_dictionary)

    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
        

/home/rlrognstad/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


NameError: name 'LDAvis_data_filepath' is not defined

In [35]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, "rb") as f:
    LDAvis_prepared = pickle.load(f)

In [36]:
pyLDAvis.display(LDAvis_prepared)

In [37]:
def get_sample_review(review_number):
    """
    retrieve a particular review index
    from the reviews file and return it
    """
    
    return list(it.islice(line_review(review_txt_filepath),
                          review_number, review_number+1))[0]

In [38]:
def lda_description(review_text, min_topic_freq=0.05):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """
    
    # parse the review text with spaCy
    parsed_review = nlp(review_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_review = [token.lemma_ for token in parsed_review
                      if not punct_space(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_review = bigram_model[unigram_review]
    trigram_review = trigram_model[bigram_review]
    
    # remove any remaining stopwords
    trigram_review = [term for term in trigram_review
                      if not term in spacy.lang.en.stop_words.STOP_WORDS]
    
    # create a bag-of-words representation
    review_bow = trigram_dictionary.doc2bow(trigram_review)
    
    # create an LDA representation
    review_lda = lda[review_bow]
    
    # sort with the most highly related topics first
    review_lda = sorted(review_lda, key=lambda review_lda: -review_lda[1])
                       
    for topic_number, freq in review_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print('{:25} {}'.format(topic_names[topic_number],
                                round(freq, 5)))

In [47]:
sample_review = get_sample_review(20000)
print(sample_review)

Amazing! This is what a happy and delicious dining experience should be. 

The restaurant is casual and comfortable. The staff is genuinely friendly, enthusiastic and consistently attentive. The pasta is sooo fresh: melty, chewy and pillow-y at once. Ordered the orecchiette with sausage, mushrooms and broccolini. It was bursting with such intense and savory flavors and texture that I rolled my eyes back in delight. I was even compelled to have a glass a wine when I finished the leftovers for lunch the next day, just to keep the memory alive...

Also ordered the chicken parm, which was huge, homey and satisfying. We watched a variety of dishes go by to other smiling patrons and heard them discussing how tasty their food was. They make their own sausage, so don't deny yourself the opportunity! Dessert, espresso and cappuccino was on par with everything else. We had lots of leftovers and had to go the whole distance. 

Yes, there is an abundance of Italian restaurants in town, but Parma h

In [48]:
lda_description(sample_review)

good taste                0.18612000346183777
experience                0.1827400028705597
italian                   0.1476300060749054
high end                  0.10487999767065048
good service              0.08748999983072281
wine & dine               0.059700001031160355
coffee shop               0.05861999839544296


/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [49]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = 'word2vec_model_all_eats_30'

In [ ]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the word2vec model yourself.
if 1 == 1:

    # initiate the model and perform the first epoch of training
    food2vec = Word2Vec(trigram_sentences, size=100, window=5,
                        min_count=20, sg=1, workers=4)
    
    food2vec.save(word2vec_filepath)

    # perform another 11 epochs of training
    for i in range(1,12):

        food2vec.train(trigram_sentences,total_examples=food2vec.corpus_count, epochs = i)
        food2vec.save(word2vec_filepath)
        print(i)
# load the finished model from disk
#food2vec = Word2Vec.load(word2vec_filepath)
#food2vec.init_sims()

#print(u'{} training epochs so far.'.format(food2vec.train_count))

/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/utils.py:568: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  if hasattr(self, attrib):
/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/utils.py:569: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  asides[attrib] = getattr(self, attrib)
/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/utils.py:570: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  delattr(self, attrib)
/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/utils.py:525: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  setattr(obj, attrib, val)
/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/u

In [104]:
food2vec.train(trigram_sentences,total_examples=food2vec.corpus_count, epochs = 12)
food2vec.save(word2vec_filepath)
print(12)

/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/utils.py:568: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  if hasattr(self, attrib):
/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/utils.py:569: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  asides[attrib] = getattr(self, attrib)
/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/utils.py:570: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  delattr(self, attrib)


12


/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/utils.py:525: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  setattr(obj, attrib, val)


In [50]:
food2vec = Word2Vec.load(word2vec_filepath)
food2vec.init_sims()

print(u'{} training epochs so far.'.format(food2vec.train_count))

/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/utils.py:461: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  setattr(self, attrib, None)


In [51]:
print(u'{} training epochs so far.'.format(food2vec.train_count))

9 training epochs so far.


In [53]:
vars(food2vec)

{'__ignoreds': ['cum_table'],
 '__numpys': [],
 '__recursive_saveloads': ['wv', 'vocabulary', 'trainables'],
 '__scipys': [],
 'alpha': 0.025,
 'batch_words': 10000,
 'callbacks': (),
 'cbow_mean': 1,
 'compute_loss': False,
 'corpus_count': 19183168,
 'epochs': 8,
 'hs': 0,
 'load': <function gensim.utils.call_on_class_only>,
 'min_alpha': 0.0001,
 'min_alpha_yet_reached': 0.00010010676156305132,
 'model_trimmed_post_training': False,
 'negative': 5,
 'random': <mtrand.RandomState at 0x7fd799ea6750>,
 'running_training_loss': 0.0,
 'sg': 1,
 'total_train_time': 13499.781131293144,
 'train_count': 9,
 'trainables': <gensim.models.word2vec.Word2VecTrainables at 0x7fd73a850e10>,
 'vector_size': 100,
 'vocabulary': <gensim.models.word2vec.Word2VecVocab at 0x7fd73a8505c0>,
 'window': 5,
 'workers': 4,
 'wv': <gensim.models.keyedvectors.Word2VecKeyedVectors at 0x7fd79b80b438>}

In [64]:
print(u'{:,} terms in the food2vec vocabulary.'.format(len(food2vec.wv.vocab)))

75,694 terms in the food2vec vocabulary.


In [80]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in food2vec.wv.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda ordered_vocab: -ordered_vocab[2])

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the food2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(food2vec.wv.syn0norm[term_indices, :],
                           index=ordered_terms)

word_vectors

/home/rlrognstad/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
-PRON-,0.047427,-0.036706,-0.039594,0.028547,-0.022271,-0.030580,0.010191,-0.056389,0.002470,-0.006041,...,-0.137453,-0.136811,0.125398,0.017733,-0.042235,0.140938,-0.092936,0.020482,-0.039396,-0.035288
be,-0.007809,0.084593,0.125174,-0.042724,0.044951,0.092232,-0.090610,0.022608,-0.038081,-0.148648,...,-0.115066,0.027881,0.176999,0.135683,-0.037958,0.034219,0.021779,-0.023820,0.081310,-0.207994
the,0.026969,0.018798,0.114487,-0.124986,0.060800,0.040325,-0.079534,-0.034554,-0.044160,-0.090763,...,-0.002141,-0.106688,0.093983,0.042477,-0.103217,-0.009616,-0.129634,0.140172,-0.059706,-0.043835
and,-0.024731,0.118873,0.077630,0.006317,0.098264,-0.075971,-0.088399,-0.002237,-0.025246,0.034113,...,-0.128234,-0.025107,0.099224,-0.021374,-0.215479,-0.002290,0.011504,0.103476,0.119236,-0.151313
a,0.005298,-0.115642,0.200763,0.067013,0.033169,0.139194,-0.121968,0.013965,0.164529,0.051003,...,-0.166060,-0.004954,0.036482,-0.048969,-0.169605,-0.022257,-0.207310,0.161581,0.014712,-0.028542
to,0.029436,0.078657,-0.007259,-0.126688,-0.147041,-0.119470,0.017709,-0.002541,-0.159055,0.278252,...,-0.126704,-0.145380,0.112392,0.114624,-0.159710,0.115735,0.009496,-0.007999,-0.020377,-0.086368
of,0.088319,-0.100087,0.258144,-0.005873,-0.065580,-0.025260,0.069144,-0.200034,0.015518,-0.022048,...,-0.121486,-0.039179,0.074309,-0.072487,0.054125,0.063298,0.000632,0.090019,0.103139,-0.019056
have,-0.111769,-0.090305,0.036318,-0.104526,0.110714,-0.097787,0.140101,-0.141873,0.084538,-0.012670,...,-0.067750,-0.085987,0.145872,0.067907,0.020362,0.105971,-0.105800,0.015717,0.105440,-0.014291
not,0.007580,-0.193473,-0.015811,0.026513,-0.064386,-0.040225,0.002773,-0.128376,-0.007547,-0.070837,...,0.047412,0.045773,0.042340,0.198485,-0.113815,0.037092,0.080811,0.096690,-0.063759,-0.211751
for,-0.081402,0.005343,-0.041587,-0.242453,0.048606,0.016759,0.109921,-0.048449,0.058966,0.228709,...,-0.032770,-0.102519,0.065942,0.113712,-0.056545,0.042570,0.018497,0.016096,0.049234,0.063431


In [81]:
def get_related_terms(token, topn=10):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in food2vec.most_similar(positive=[token], topn=topn):

        print(u'{:20} {}'.format(word, round(similarity, 3)))

In [86]:
get_related_terms(u'beer')

draft_beer           0.872
craft_beer           0.87
microbrew            0.863
draft                0.856
draught              0.855
micro_brew           0.841
beers                0.837
craft_brew           0.837
bottled_beer         0.83
draught_beer         0.829


/home/rlrognstad/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


In [89]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = food2vec.wv.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print(term)

In [90]:
word_algebra(add=[u'breakfast', u'lunch'])

brunch


In [91]:
word_algebra(add=[u'beer', u'fancy'])

microbrew


In [95]:
word_algebra(add=[u'wine', u'barley'], subtract=[u'grapes'])

kronenbourg


In [92]:
word_algebra(add=[u'coffee', u'snack'], subtract=[u'drink'])

bakery


In [100]:
word_algebra(add=[u'beef', u'cheese'])

mushroom
